In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import itertools as it

In [7]:
data = pd.read_csv('movie_bd_v5.csv')
# первично изучаем данные, с которыми предстоит работать
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 140.3+ KB


In [8]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [9]:
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
13,tt2120120,88000000,243637091,Pixels,Adam Sandler|Michelle Monaghan|Peter Dinklage|...,Chris Columbus,Game On.,Video game experts are recruited by the milita...,105,Action|Comedy|Science Fiction,Columbia Pictures|Happy Madison Productions,7/16/2015,5.8,2015
1371,tt0408306,70000000,130358911,Munich,Eric Bana|Daniel Craig|CiarÃ¡n Hinds|Mathieu K...,Steven Spielberg,The world was watching in 1972 as 11 Israeli a...,"During the 1972 Olympic Games in Munich, eleve...",164,Drama|Action|History|Thriller,DreamWorks SKG|Universal Pictures|Amblin Enter...,12/22/2005,6.7,2005
1289,tt0795461,20000000,78378744,Scary Movie 5,Simon Rex|Ashley Tisdale|Charlie Sheen|Lindsay...,Malcolm D. Lee,Evil is coming. Bring protection.,"Home with their newly-formed family, happy par...",86,Comedy,Dimension Films,4/11/2013,4.6,2013
264,tt0327597,60000000,124596398,Coraline,Dakota Fanning|Teri Hatcher|Jennifer Saunders|...,Henry Selick,Be careful what you wish for.,"When Coraline moves to an old house, she feels...",100,Animation|Family,Focus Features|Laika Entertainment,2/5/2009,7.1,2009
325,tt1045778,60000000,62357900,Year One,Jack Black|Michael Cera|Olivia Wilde|Oliver Pl...,Harold Ramis,Meet your ancestors,When a couple of lazy hunter-gatherers are ban...,97,Comedy|Adventure,Columbia Pictures|Ocean Pictures|Apatow Produc...,6/18/2009,4.5,2009


# Предобработка

In [10]:
# добавим колонки "прибыль", "длина заголовка", "количество слов в описании"
data['profit'] = data['revenue'] - data['budget']
data['length_title'] = data.original_title.str.len()
data['count_words_overview'] = data.overview.str.split(' ').str.len()

# переведём строковый формат даты релиза в более удобный и добавим колонку "месяц релиза"
data.release_date = pd.to_datetime(data.release_date)
data['release_month'] = data.release_date.dt.month

#создадим функцию, которая сократит код и ускорит поиск ответов на вопросы викторины
def data_exploded (column):
    "Функция сплита строковых значений указанной колонки и создания нового DataFrame с использованием метода explode"
    new_data = data.copy()
    new_data[column] = new_data[column].str.split('|')
    return new_data.explode(column)

# создадим словарь для ответов
answers = {} 

# 1. У какого фильма из списка самый большой бюджет?

In [11]:
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides	(tt1298650)' # "+"

In [12]:
data[data.budget == data.budget.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


# 2. Какой из фильмов самый длительный (в минутах)?

In [13]:
answers['2'] = '2. Gods and Generals (tt0279111)' # "+"

In [14]:
data[data.runtime == data.runtime.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
1157,Gods and Generals,tt0279111


# 3. Какой из фильмов самый короткий (в минутах)?





In [15]:
answers['3'] = '3. Winnie the Pooh (tt1449283)' # "+"

In [16]:
data[data.runtime == data.runtime.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
768,Winnie the Pooh,tt1449283


# 4. Какова средняя длительность фильмов?


In [17]:
answers['4'] = '2. 110' # "+"

In [18]:
# окрулим среднюю длительность фильмов до целых
round(data.runtime.mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [19]:
answers['5'] = '1. 107' # "+"

In [20]:
round(data.runtime.median())

107

# 6. Какой самый прибыльный фильм?

In [21]:
answers['6'] = '5. Avatar (tt0499549) ' # "+"

In [22]:
data[data.profit == data.profit.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
239,Avatar,tt0499549


# 7. Какой фильм самый убыточный? 

In [23]:
answers['7'] = '5. The Lone Ranger (tt1210819)' # "+"

In [24]:
data[data.profit == data.profit.min()][['original_title', 'imdb_id']]

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [25]:
answers['8'] = '1. 1478' # "+"

In [26]:
data.query('profit > 0').count()[0]

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [27]:
answers['9'] = '4. The Dark Knight (tt0468569)' # "+"

In [28]:
data[data.release_year == 2008].sort_values(by = 'revenue', ascending = False)[['original_title', 'imdb_id']].iloc[0]

original_title    The Dark Knight
imdb_id                 tt0468569
Name: 599, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [29]:
answers['10'] = '5. The Lone Ranger (tt1210819)' # "+"

In [30]:
data.query('2012 <= release_year <= 2014').sort_values(by = 'profit')[['original_title', 'imdb_id']].head(1)

,original_title,imdb_id
1245,The Lone Ranger,tt1210819


# 11. Какого жанра фильмов больше всего?

In [31]:
answers['11'] = '3. Drama' # "+"

In [32]:
data_exploded('genres').genres.value_counts().head(1)

Drama    782
Name: genres, dtype: int64


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [33]:
answers['12'] = '1. Drama' # "+"

In [34]:
data_exploded('genres').query('profit > 0').genres.value_counts().head(1)

Drama    560
Name: genres, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [35]:
answers['13'] = '5. Peter Jackson ' # "+"

In [36]:
data_exploded ('director').groupby(['director']).sum().sort_values(by ='revenue', ascending = False).head(1)

,budget,revenue,runtime,vote_average,release_year,profit,length_title,count_words_overview,release_month
director,,,,,,,,,
Peter Jackson,1288000000,6490593685,1355,58.1,16059,5202593685,265,363,95


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [37]:
answers['14'] = '3. Robert Rodriguez' # "+"

In [38]:
data_exploded ('director')[data_exploded ('director').genres.str.contains('Action', na = False)].director.value_counts().head(1)

Robert Rodriguez    9
Name: director, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [39]:
answers['15'] = '3. Chris Hemsworth ' # "+"

In [40]:
data_exploded ('cast').query('release_year == 2012').groupby(['cast']).sum().sort_values(by= 'revenue', ascending=False).head(1)

,budget,revenue,runtime,vote_average,release_year,profit,length_title,count_words_overview,release_month
cast,,,,,,,,,
Chris Hemsworth,485000000,2027450773,458,24.8,8048,1542450773,69,253,24


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [41]:
answers['16'] = '3. Matt Damon' # "+"

In [42]:
data_exploded('cast').query('budget > budget.mean()').cast.value_counts().sort_values(ascending=False).head(1)

Matt Damon    18
Name: cast, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [43]:
answers['17'] = '2. Action' # "+"

In [44]:
data_exploded('genres')[data_exploded('genres').cast.str.contains("Nicolas Cage", na = False)].genres.value_counts().head(1)

Action    17
Name: genres, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [45]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)' # "+"

In [46]:
data[data['production_companies'].str.contains('Paramount Pictures', na=False)].sort_values(by = 'profit', ascending = True).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,length_title,count_words_overview,release_month
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,2002-07-19,6.0,2002,-64831034,20,23,7


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [47]:
answers['19'] = '5. 2015' # "+"

In [48]:
data.groupby(['release_year']).revenue.sum().sort_values(ascending = False).head(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [49]:
answers['20'] = '1. 2014' # "+"

In [50]:
data[data['production_companies'].str.contains('Warner Bros', na = False)].groupby(['release_year']).profit.sum().sort_values(ascending = False).head(1)

release_year
2014    2295464519
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [51]:
answers['21'] = '4. Сентябрь' # "+"

In [52]:
data.release_month.value_counts().sort_values(ascending=False).head(1)

9    227
Name: release_month, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [53]:
answers['22'] = '2. 450' # "+"

In [54]:
data.query('6 <=release_month <= 8').count()[0]

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [55]:
answers['23'] = '5. Peter Jackson ' # "+"

In [56]:
data_exploded ('director').query('1 <= release_month <= 2 | release_month == 12').director.value_counts().head(1)

Peter Jackson    7
Name: director, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [57]:
answers['24'] = '5. Four By Two Productions' # "+"

In [58]:
data_exploded ('production_companies').groupby('production_companies')['length_title'].mean().sort_values(ascending = False).head(1)

production_companies
Four By Two Productions    83.0
Name: length_title, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [59]:
answers['25'] = '3. Midnight Picture Show верно' # "+"

In [60]:
data_exploded ('production_companies').groupby('production_companies')['count_words_overview'].mean().sort_values(ascending = False).head(1)

production_companies
Midnight Picture Show    175.0
Name: count_words_overview, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 

In [61]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave ' # "+"

In [62]:
#вручную сопоставим 1% лучших по рейтингу фильмов с вариантами из викторины
data[data.vote_average > np.percentile(data.vote_average, 99)].original_title

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [63]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint ' # "+"

In [64]:
# создаём копию DataFrame
data27 = data.copy()
# переводим значения колонки cast в итерируемые списки
data27['cast'] = data27.cast.str.split('|')
# из итерируемых списков получаем кортежи с уникальными парами актёров
data27.cast = data27.cast.apply(lambda x: list(it.combinations(sorted(x), 2)))
# получаем самые распространённые пары
data27.explode('cast').cast.mode()

0     (Daniel Radcliffe, Emma Watson)
1    (Daniel Radcliffe, Rupert Grint)
2         (Emma Watson, Rupert Grint)
dtype: object

# Submission

In [65]:
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides\t(tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549) ',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson ',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth ',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson ',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show верно',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave ',
 '27': '5. Daniel Radcliffe & Rupert Grint '}

In [66]:
len(answers)

27

In [67]:
#Вывод: удалось ответить на все вопросы викторины